In [2]:
# -*- coding: utf-8 -*-
# @Author   : ly
# @Time     : 2022/7/2 15:49
# @File     : tushare_token.py
# @Project  : ai_quant_trade
# Copyright (c) Personal 2022 LY
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 1. 安装和简介
Tushare网址: [**https://tushare.pro/**](https://tushare.pro/)
安装
``` sh
pip install tushare
```
**注：如果大量调用，或需要获取更详细的信息，需要付费**
- 旧版本接口将不再维护，建议使用新接口Tushare Pro (需要注册获取token)
- 注册后在右上角用户头像--》个人主页--》获取token
- 如果没有缴费加入会员，很多接口都无法调用(每个接口调用需要的积分数，请查看接口文档)

# 2. TuShare数据
## 2.1 从tushare_pro获取数据

In [3]:
import tushare as ts
import datetime
# replace below with your token and comment my import
# tushare_token = 'xxxx'
from data.private.tushare_token import tushare_token

# initialize api
ts.set_token(tushare_token)
ts_pro = ts.pro_api()

# query date setting
# date type is year-month-day
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)
five_days_ago = today - datetime.timedelta(days = 5)
# change date type to YearMonthDay
yesterday = yesterday.strftime("%Y%m%d")
five_days_ago = five_days_ago.strftime("%Y%m%d")

# request data，中国平安: 601318.SH, date type require is: YearMonthDay
# api website: https://tushare.pro/document/2?doc_id=27
# !!! 警告：未复权，如果是复权请用 通用行情接口
df = ts_pro.query('daily', ts_code='601318.SH',
                    start_date=five_days_ago, end_date=yesterday)
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,601318.SH,20220809,41.17,41.21,40.92,40.96,41.18,-0.22,-0.5342,251397.03,1031147.757
1,601318.SH,20220808,41.30,41.54,41.00,41.18,41.54,-0.36,-0.8666,301594.03,1242870.500
2,601318.SH,20220805,41.29,41.68,40.98,41.54,41.08,0.46,1.1198,296710.19,1224704.136


explanation of each column above

| 名称 |	类型 | 描述 |
|:-------- |:-------:| ---------:|
| ts_code |str | 股票代码 |
| trade_date | str | 交易日期 |
| open | float | 开盘价 |
| high | float | 最高价 |
| low | float | 最低价 |
| close | float | 收盘价 |
| pre_close | float | 昨收价 |
| change | float | 涨跌额 |
| pct_chg | float | 涨跌幅 （未复权，如果是复权请用 通用行情接口 ）
| vol | float | 成交量 （手） |
| amount| float |成交额 （千元）|

## 2.2 从老接口获取数据
此接口不稳定，连接时间有限，且不在维护

In [4]:
import tushare as ts
# 请勿配置token，否则如下api无法被识别调用，会被当做pro的接口
# ktype: 5,15, 30, 60分钟
df_old = ts.get_hist_data(code='601318.SH',
                    start=five_days_ago, end=yesterday, ktype='60')
df_old.head()

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


AttributeError: 'NoneType' object has no attribute 'head'

# 3. 其它常用接口
## 3.1 股票列表

In [6]:
# #查询当前所有正常上市交易的股票列表
# （注册后修改个人信息）即可免费调取
# list_status: 上市状态 L上市 D退市 P暂停上市，默认是L
# exchange: 交易所 SSE上交所 SZSE深交所 BSE北交所
# market: 市场类别 （主板/创业板/科创板/CDR/北交所）
data = ts_pro.query('stock_basic', exchange='SSE', market='主板',
                 list_status='L', fields='ts_code,symbol,name,area,industry,market,list_date')
data.head()

,ts_code,symbol,name,area,industry,market,list_date
0,600000.SH,600000,浦发银行,上海,银行,主板,19991110
1,600004.SH,600004,白云机场,广东,机场,主板,20030428
2,600006.SH,600006,东风汽车,湖北,汽车整车,主板,19990727
3,600007.SH,600007,中国国贸,北京,园区开发,主板,19990312
4,600008.SH,600008,首创环保,北京,环境保护,主板,20000427


In [ ]:
print(len(data))

## 3.2 大盘指数获取

In [ ]:
# benchmark to get market index,
#  000001.XSHG: Shanghai securities composite index
#  000300.XSHG: Shanghai and Shenzhen 300 index

# 权限：用户累积2000积分可调取，5000积分以上频次相对较高
# get index data
df = ts_pro.index_daily(ts_code='000001.XSHG',
                        start_date='2022-08-01', end_date='2022-08-10')

# 3.3 股票技术因子（量化因子）

In [5]:
# https://tushare.pro/document/2?doc_id=296
df1 = ts_pro.stk_factor(ts_code='600000.SH', start_date='20220501', end_date='20220520',
                    fields='ts_code,trade_date,macd,kdj_k,kdj_d,kdj_j')
df1.head()

,ts_code,trade_date,macd,kdj_k,kdj_d,kdj_j
0,600000.SH,20220520,0.026,72.966,64.718,89.460
1,600000.SH,20220519,0.015,63.615,60.594,69.656
2,600000.SH,20220518,0.022,67.645,59.084,84.766
3,600000.SH,20220517,0.024,68.134,54.804,94.794
4,600000.SH,20220516,0.013,60.309,48.139,84.649


注：
1、本接口的前复权行情是从最新一个交易日开始往前复权，跟行情软件一致。
2、pro_bar接口的前复权是动态复权，即以end_date参数开始往前复权，与本接口会存在不一致的可能，属正常。
3、本接口技术指标都是基于前复权价格计算。

输入参数

名称	类型	必选	描述
ts_code	str	N	股票代码
trade_date	str	N	交易日期 （yyyymmdd，下同）
start_date	str	N	开始日期
end_date	str	N	结束日期


输出参数

名称	类型	默认显示	描述
ts_code	str	Y	股票代码
trade_date	str	Y	交易日期
close	float	Y	收盘价
open	float	Y	开盘价
high	float	Y	最高价
low	float	Y	最低价
pre_close	float	Y	昨收价
change	float	Y	涨跌额
pct_change	float	Y	涨跌幅
vol	float	Y	成交量 （手）
amount	float	Y	成交额 （千元）
adj_factor	float	Y	复权因子
open_hfq	float	Y	开盘价后复权
open_qfq	float	Y	开盘价前复权
close_hfq	float	Y	收盘价后复权
close_qfq	float	Y	收盘价前复权
high_hfq	float	Y	最高价后复权
high_qfq	float	Y	最高价前复权
low_hfq	float	Y	最低价后复权
low_qfq	float	Y	最低价前复权
pre_close_hfq	float	Y	昨收价后复权
pre_close_qfq	float	Y	昨收价前复权
macd_dif	float	Y	MCAD_DIF (基于前复权价格计算，下同)
macd_dea	float	Y	MCAD_DEA
macd	float	Y	MCAD
kdj_k	float	Y	KDJ_K
kdj_d	float	Y	KDJ_D
kdj_j	float	Y	KDJ_J
rsi_6	float	Y	RSI_6
rsi_12	float	Y	RSI_12
rsi_24	float	Y	RSI_24
boll_upper	float	Y	BOLL_UPPER
boll_mid	float	Y	BOLL_MID
boll_lower	float	Y	BOLL_LOWER
cci	float	Y	CCI